In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Step 01: Loading the UCS Satellite Dataset

In [10]:
file_path= "../data/text-data/ucs_satellites.txt"

df= pd.read_csv(file_path, sep="\t", encoding="latin1")

print("Shape of the dataset: ", df.shape)
print("\nColumn names: ", df.columns)
print("\nFirst 5 rows: ", df.head())
print("\nData types: ", df.dtypes)


Shape of the dataset:  (7562, 67)

Column names:  Index(['Current Official Name of Satellite', 'Country/Org of UN Registry',
       'Country of Operator/Owner', 'Operator/Owner', 'Users', 'Purpose',
       'Detailed Purpose', 'Class of Orbit', 'Type of Orbit',
       'Longitude of GEO (degrees)', 'Perigee (km)', 'Apogee (km)',
       'Eccentricity', 'Inclination (degrees)', 'Period (minutes)',
       'Launch Mass (kg.)', ' Dry Mass (kg.) ', 'Power (watts)',
       'Date of Launch', 'Expected Lifetime (yrs.)', 'Contractor',
       'Country of Contractor', 'Launch Site', 'Launch Vehicle',
       'COSPAR Number', 'NORAD Number', 'Comments', 'Unnamed: 27',
       'Source Used for Orbital Data', 'Source', 'Source.1', 'Source.2',
       'Source.3', 'Source.4', 'Source.5', 'Source.6', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40',
       'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44',
       'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 

# Step 02: Cleaning Dataset
- Removing empty or irrelevant columns (like "Unnamed: XX").
- Standardizing column names (strip spaces, lowercase, replace spaces with underscores).

In [11]:
df= df.loc[:, ~df.columns.str.contains('^Unnamed')]

df.columns= (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r'[^\w]', '', regex=True)
)

print("Shape after cleaning: ", df.shape)
print("\nColumn names after cleaning: \n", df.columns.tolist()[:20])


Shape after cleaning:  (7562, 35)

Column names after cleaning: 
 ['current_official_name_of_satellite', 'countryorg_of_un_registry', 'country_of_operatorowner', 'operatorowner', 'users', 'purpose', 'detailed_purpose', 'class_of_orbit', 'type_of_orbit', 'longitude_of_geo_degrees', 'perigee_km', 'apogee_km', 'eccentricity', 'inclination_degrees', 'period_minutes', 'launch_mass_kg', 'dry_mass_kg', 'power_watts', 'date_of_launch', 'expected_lifetime_yrs']


# Step 03: Handling Missing Values and Selecting Core Columns
- Identifying and handling missing values (e.g., filling with default values, removing rows/columns).
- Selecting only the core columns that are essential for analysis (e.g., "satellite_name", "launch_date", "country", "payload_mass_kg", "orbit_type").

In [12]:
missing_values= df.isnull().sum().sort_values(ascending=False)
print("\nMissing values in each column: \n", missing_values.head(20))

core_columns = [
    "current_official_name_of_satellite",
    "countryorg_of_un_registry",
    "country_of_operatorowner",
    "operatorowner",
    "users",
    "purpose",
    "class_of_orbit",
    "type_of_orbit",
    "perigee_km",
    "apogee_km",
    "inclination_degrees",
    "period_minutes",
    "launch_mass_kg",
    "power_watts",
    "date_of_launch",
    "expected_lifetime_yrs"
]

df_core= df[[col for col in core_columns if col in df.columns]]

print("\nShape of reduced dataset:", df_core.shape)
print("\nFirst 5 rows:\n", df_core.head())


Missing values in each column: 
 source6                         7058
source5                         7009
power_watts                     6983
source1                         6837
source4                         6833
dry_mass_kg                     6795
source3                         6436
detailed_purpose                6308
source2                         5730
comments                        5477
source                          4276
expected_lifetime_yrs           2112
source_used_for_orbital_data     925
type_of_orbit                    653
launch_mass_kg                   247
period_minutes                    58
eccentricity                      13
apogee_km                          9
perigee_km                         9
inclination_degrees                6
dtype: int64

Shape of reduced dataset: (7562, 16)

First 5 rows:
   current_official_name_of_satellite countryorg_of_un_registry  \
0                         1HOPSAT-TD                        NR   
1            AAC AIS-Sat1Ê(

# Step 04: Data Type Conversion
- Converting columns to appropriate data types (e.g., dates, numerical values).


In [13]:
df_core['date_of_launch']= pd.to_datetime(df_core['date_of_launch'], errors='coerce')

numeric_columns= [
    "perigee_km",
    "apogee_km",
    "inclination_degrees",
    "period_minutes",
    "launch_mass_kg",
    "power_watts",
    "expected_lifetime_yrs"
]

for col in numeric_columns:
    if col in df_core.columns:
        df_core[col]= pd.to_numeric(df_core[col], errors='coerce')

print("\nDataset info:")
print(df_core.info())

print("\nSummary stats for numeric columns:\n", df_core.describe().T)

print("\nFirst 5 rows after wrangling:\n", df_core.head())


Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7562 entries, 0 to 7561
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   current_official_name_of_satellite  7560 non-null   object        
 1   countryorg_of_un_registry           7559 non-null   object        
 2   country_of_operatorowner            7560 non-null   object        
 3   operatorowner                       7560 non-null   object        
 4   users                               7560 non-null   object        
 5   purpose                             7560 non-null   object        
 6   class_of_orbit                      7560 non-null   object        
 7   type_of_orbit                       6909 non-null   object        
 8   perigee_km                          6210 non-null   float64       
 9   apogee_km                           6185 non-null   float64       
 10  inclinati

C:\Users\asus\AppData\Local\Temp\ipykernel_21744\3599063134.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_core['date_of_launch']= pd.to_datetime(df_core['date_of_launch'], errors='coerce')
C:\Users\asus\AppData\Local\Temp\ipykernel_21744\3599063134.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_core['date_of_launch']= pd.to_datetime(df_core['date_of_launch'], errors='coerce')
C:\Users\asus\AppData\Local\Temp\ipykernel_21744\3599063134.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 